In [15]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

In [16]:
# загружаем исходные данные
df = pd.read_csv('6_result.csv')
df.head(3)

,Name,normalised_name,raw_value,mol_file
0,As20S60Ge20,"[['As', 20.0], ['Ge', 20.0], ['S', 60.0]]",2.054,NaN
1,PEO,OO,1.470,<rdkit.Chem.rdchem.Mol object at 0x0000025F21E...
2,Sapphire,O=[Al]O[Al]=O,1.750,<rdkit.Chem.rdchem.Mol object at 0x0000025F21E...


In [ ]:
# добавляем в датафрем колонку mol файлов, для тех соединений, для которых это возможно сделать
mols = [Chem.MolFromSmiles(smi) for smi in df['normalised_name'].tolist()]
df['mol_file'] = mols

In [18]:
# разбиваем датафрейм на части, которые содержат mol file и None
df_nans = df.loc[df.mol_file.isna()]
df_yes = df.loc[df.mol_file.notna()]

In [19]:
# количество дескрипторов 
print(len(Descriptors._descList))

208


In [20]:
# зададим вычислитель дескрипторов
calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])

In [21]:
# имена дескрипторов
desc_names = calc.GetDescriptorNames()

In [ ]:
# выгрузим дескрипторы и создадим датафрейм
mol_descriptors = []

for mol in df_yes['mol_file']:
        descriptors = calc.CalcDescriptors(mol)
        mol_descriptors.append(descriptors)

df_molecular_desc = pd.DataFrame(mol_descriptors, columns=desc_names)
df_molecular_desc.shape

In [23]:
# объединяем датафреймы
df_with_desc = pd.concat([df_yes.reset_index(drop=True), df_molecular_desc], axis=1)

In [26]:
df_result = pd.concat([df_with_desc, df_nans.reset_index(drop=True)])

In [28]:
df_result.to_csv('db_descriptors_RDKit.csv', index=False)